#### This notebook analyses the horizontally-averaged vertical heat budget in the abrupt ACCESS-OM2 experiments.

In [6]:
import cosima_cookbook as cc
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cmocean as cm
from scipy.optimize import curve_fit
from xhistogram.xarray import histogram
from dask.distributed import Client
from collections import OrderedDict


%matplotlib inline

In [2]:
client = Client(n_workers=8)
display(client)

Client Scheduler: tcp://127.0.0.1:44875 Dashboard: /proxy/42817/status,Cluster Workers: 8 Cores: 8 Memory: 128.85 GB


In [4]:
### Generate new session

db = '/scratch/e14/mp2135/access-om2/archive/databases/1deg_jra55_ryf_experiments_abrupt_spinup500.db'
session = cc.database.create_session(db)

cc.querying.get_experiments(session)

,experiment,ncfiles
0,1deg_jra55_ryf_cont,24764
1,1deg_jra55_ryf_rcp45warm_x1_spinup500,42426
2,1deg_jra55_ryf_rcp45cool_x1_spinup500,42426
3,1deg_jra55_ryf_warm_0sat_8dwlw_spinup500,21206
4,1deg_jra55_ryf_warm_1.5sat_0dwlw_spinup500,21206
5,1deg_jra55_ryf_RCPwarm_humid_x1,42426
6,1deg_jra55_ryf_RCPwarm_humid_x2,42207
7,1deg_jra55_ryf_RCPcool_humid_x1,42207
8,1deg_jra55_ryf_RCPcool_humid_x2,4010
9,1deg_jra55_ryf_wcwc_step_test1,2108


In [12]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

cc.querying.get_variables(session, experiment='1deg_jra55_ryf_cont')

,name,long_name,frequency,ncfile,# ncfiles,time_start,time_end
0,GRID_X_T,None,None,CHUCKABLE/restart074/ice/monthly_sstsss.nc,81,None,None
1,GRID_Y_T,None,None,CHUCKABLE/restart074/ice/monthly_sstsss.nc,81,None,None
2,GRID_Y_T_bnds,None,None,CHUCKABLE/restart074/ice/monthly_sstsss.nc,81,None,None
3,SALT,SALT,None,CHUCKABLE/restart074/ice/monthly_sstsss.nc,81,None,None
4,TEMP,TEMP,None,CHUCKABLE/restart074/ice/monthly_sstsss.nc,81,None,None
5,TMONTH,None,None,CHUCKABLE/restart074/ice/monthly_sstsss.nc,81,None,None
6,Time,Time,None,CHUCKABLE/restart044/ocean/ocean_frazil.res.nc,1106,None,None
7,Tsfcn,None,None,CHUCKABLE/restart058/ice/iced.2192-01-01-00000.nc,405,None,None
8,advectionu,advectionu,None,CHUCKABLE/restart062/ocean/ocean_velocity_adve...,79,None,None
9,advectionv,advectionv,None,CHUCKABLE/restart062/ocean/ocean_velocity_adve...,79,None,None


Exception during reset or similar
Traceback (most recent call last):
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/sqlalchemy/pool/base.py", line 697, in _finalize_fairy
    fairy._reset(pool)
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/sqlalchemy/pool/base.py", line 893, in _reset
    pool._dialect.do_rollback(self)
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/sqlalchemy/engine/default.py", line 559, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 23347140151104 and this is thread id 23344466245376.
Exception closing connection <sqlite3.Connection object at 0x153b4e1df5d0>
Traceback (most recent call last):
  File "/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.01/lib/python3.8/site-packages/sqlalchemy/po

In [13]:
yr_beg = '2400-02-01'
yr_end = '2499-12-31'

expt = '1deg_jra55_ryf_cont'
var = 'temp_int_rhodz'
temp_int_rhodz_cont = cc.querying.getvar(expt, var, session, start_time = yr_beg, end_time = yr_end)

In [14]:
temp_int_rhodz_cont

<xarray.DataArray 'temp_int_rhodz' (time: 1200, yt_ocean: 300, xt_ocean: 360)>
dask.array<concatenate, shape=(1200, 300, 360), dtype=float32, chunksize=(1, 300, 360), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * yt_ocean  (yt_ocean) float64 -77.88 -77.63 -77.38 ... 88.87 89.32 89.77
  * time      (time) object 2400-01-16 12:00:00 ... 2499-12-16 12:00:00
Attributes:
    long_name:      vertical sum of Conservative temperature * rho_dzt
    units:          deg_C*(kg/m^3)*m
    valid_range:    [-1.e+20  1.e+20]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT
    coordinates:    geolon_t geolat_t

In [8]:
exptdict = OrderedDict([
    ('Control',  {'expt':'1deg_jra55_ryf_cont'}),
    ('Warming',  {'expt':'1deg_jra55_ryf_RCPwarm_humid_x1'}),
    ('Warming x2',  {'expt':'1deg_jra55_ryf_RCPwarm_humid_x2'}),
    ('Cooling',  {'expt':'1deg_jra55_ryf_RCPcool_humid_x1'}),
    ('Cooling x2',  {'expt':'1deg_jra55_ryf_RCPcool_humid_x1'})
])
label = {'Warming': 'Warming','Warming x2': 'Warming x2','Cooling':'Cooling','Cooling x2':'Cooling x2'}

In [9]:
def vertical_heat_budget_terms(ekey):
    expt = exptdict[ekey]['expt']
    temp_tendency = cc.querying.getvar(expt, 'temp_tendency', session, ncfile = 'ocean_heat.nc')
    area_t = cc.querying.getvar(expt, 'area_t', session, ncfile = 'ocean_grid.nc', n=-1)
    dyt = cc.querying.getvar(expt, 'dyt', session, ncfile = 'ocean_grid.nc', n=-1)
    dxt= cc.querying.getvar(expt, 'dyt', session, ncfile = 'ocean_grid.nc', n=-1)
    dzt = cc.querying.getvar(expt, 'dzt', session, ncfile = 'ocean_heat.nc')
    adv = cc.querying.getvar(expt, 'temp_advection', session)
    sw_heat =  cc.querying.getvar(expt, 'sw_heat', session)
    temp_vdiffuse_diff_cbt =   cc.querying.getvar(expt, 'temp_vdiffuse_diff_cbt', session, ncfile = 'ocean_heat.nc')
    temp_submeso =   cc.querying.getvar(expt, 'temp_submeso', session, ncfile = 'ocean_heat.nc')
    mixdownslope_temp = cc.querying.getvar(expt, 'mixdownslope_temp', session, ncfile = 'ocean_heat.nc')
    neutral_gm_temp = cc.querying.getvar(expt, 'neutral_gm_temp', session, ncfile = 'ocean_heat.nc')
    temp_vdiffuse_sbc= cc.querying.getvar(expt, 'temp_vdiffuse_sbc', session, ncfile = 'ocean_heat.nc')
    temp_nonlocal_KPP = cc.querying.getvar(expt, 'temp_nonlocal_KPP', session, ncfile = 'ocean_heat.nc')
    neutral_diffusion_temp= cc.querying.getvar(expt, 'neutral_diffusion_temp', session, ncfile = 'ocean_heat.nc')
    temp_rivermix= cc.querying.getvar(expt, 'temp_rivermix', session, ncfile = 'ocean_heat.nc')
    temp_vdiffuse_k33= cc.querying.getvar(expt, 'temp_vdiffuse_k33', session, ncfile = 'ocean_heat.nc')
    frazil_3d= cc.querying.getvar(expt, 'frazil_3d', session, ncfile = 'ocean_heat.nc')
    sfc_hflux_pme = cc.querying.getvar(expt, 'sfc_hflux_pme', session, ncfile = 'ocean_heat.nc')
    temp = cc.querying.getvar(expt, 'temp', session, ncfile = 'ocean_heat.nc')
    temp_sigma_diff = cc.querying.getvar(expt, 'temp_sigma_diff', session, ncfile = 'ocean_heat.nc')
    
    advection = adv + neutral_gm_temp + temp_submeso
    vertical_mixing = temp_vdiffuse_diff_cbt + temp_nonlocal_KPP + mixdownslope_temp + temp_sigma_diff
    along_iso_mixing = neutral_diffusion_temp + temp_vdiffuse_k33
    surface_forcing = frazil_3d + temp_rivermix+ sw_heat + temp_vdiffuse_sbc
    
    new_surf = xr.concat([sfc_hflux_pme.assign_coords(st_ocean = surface_forcing.isel(st_ocean=0).st_ocean.values),(0*surface_forcing).isel(st_ocean = np.arange(1,50))], dim = 'st_ocean')
    surface_forcing_combined = surface_forcing+new_surf
    
    return temp_tendency, advection, vertical_mixing, along_iso_mixing, surface_forcing_combined, area_t

In [11]:
temp_tendency_cont, advection_cont, vertical_mixing_cont, along_iso_mixing_cont, surface_forcing_combined_cont, area_t = vertical_heat_budget_terms('Control')

VariableNotFoundError: No files were found containing 'temp_tendency' in the '1deg_jra55_ryf_cont' experiment